In [2]:
import csv
from nltk.tokenize import word_tokenize
import nltk
import numpy
import random

In [3]:
all_tweets = []
with open('realDonaldTrump_tweets.csv', 'r', encoding = 'utf8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        all_tweets.append(row)

In [4]:
all_tweets_separate = []
for tweet in all_tweets:
    for words in tweet:
        #print(words + ' loop 1')
        for word in words.split():
            #print(word +' loop 2' )
            if "://" not in word:
                all_tweets_separate.append(word) 
        all_tweets_separate.append(" . ")

In [5]:
all_tweets = list(filter(lambda x : len(x) != 0, all_tweets))

In [6]:
text = " ".join(all_tweets_separate)

In [7]:
text = word_tokenize(text)
pos_tags = nltk.pos_tag(text)

In [8]:
index = 0
posDict = {}
posDictReverse = {}
for word, pos in pos_tags:
    if pos not in posDict:
        posDict[pos] = index
        posDictReverse[index] = pos
        index +=1

In [9]:
transition_prob = numpy.zeros((len(posDict), len(posDict)))

In [10]:
for i in range(1,len(pos_tags)):
    transition_prob[posDict[pos_tags[i-1][1]]][posDict[pos_tags[i][1]]] += 1

In [11]:
matSum = []
for i in range(0, len(transition_prob)):
    matSum.append(sum(transition_prob[i]))
for i in range(0, len(transition_prob)):
    transition_prob[i] = transition_prob[i] / matSum[i]
    

In [12]:
index = 0
wordDict = {}
wordDictReverse = {}
for word, pos in pos_tags:
    if word not in wordDict:
        wordDict[word] = index
        wordDictReverse[index] = word
        index +=1

In [13]:
emmision_prob = numpy.zeros((len(posDict), len(wordDict)))

In [14]:
for i in range(0, len(pos_tags)):
    emmision_prob[posDict[pos_tags[i][1]]][wordDict[pos_tags[i][0]]] += 1

In [15]:
matSum = []
for i in range(0, len(emmision_prob)):
    matSum.append(sum(emmision_prob[i]))
for i in range(0, len(emmision_prob)):
    emmision_prob[i] = emmision_prob[i] / matSum[i]
    

In [16]:
phrase = 'Lukaku secures a goal'

In [17]:
text = word_tokenize(phrase)
text = nltk.pos_tag(text)
text

[('Lukaku', 'NNP'), ('secures', 'VBZ'), ('a', 'DT'), ('goal', 'NN')]

In [18]:
def place_before(word, pos):
    rand_num = random.random()
    if random.random() > 0.5:
        rand_num = random.random()
        possibilities = numpy.transpose(transition_prob[posDict[pos]])
        for i in range(0, len(possibilities)):
            if (rand_num-possibilities[i]) > 0:
                rand_num -= possibilities[i]
            else:
                break
        next_pos = posDictReverse[i]
        rand_num = random.random()
        possibilities = emmision_prob[posDict[next_pos]]
        for i in range(0, len(possibilities)):
            if (rand_num-possibilities[i]) > 0:
                rand_num -= possibilities[i]
            else:
                break
        next_word = wordDictReverse[i]
        return next_word
    else:
        return ""
    
    

In [19]:
def place_after(word, pos):
    if random.random() > 0.5:
        rand_num = random.random()
        possibilities = transition_prob[posDict[pos]]
        for i in range(0, len(possibilities)):
            if (rand_num-possibilities[i]) > 0:
                rand_num -= possibilities[i]
            else:
                break
        next_pos = posDictReverse[i]
        rand_num = random.random()
        possibilities = emmision_prob[posDict[next_pos]]
        for i in range(0, len(possibilities)):
            if (rand_num-possibilities[i]) > 0:
                rand_num -= possibilities[i]
            else:
                break
        next_word = wordDictReverse[i]
        return next_word
    else:
        return ''   

In [20]:
def add_word(word, pos):
    if word == '':
        rand_num = random.random()
        possibilities = transition_prob[posDict['.']]
        for i in range(0, len(possibilities)):
            if (rand_num-possibilities[i]) > 0:
                rand_num -= possibilities[i]
            else:
                break
        next_pos = posDictReverse[i]
        rand_num = random.random()
        possibilities = emmision_prob[posDict[next_pos]]
        for i in range(0, len(possibilities)):
            if (rand_num-possibilities[i]) > 0:
                rand_num -= possibilities[i]
            else:
                break
        next_word = wordDictReverse[i]
        return [next_word, next_pos]
    else:
        rand_num = random.random()
        possibilities = transition_prob[posDict[pos]]
        for i in range(0, len(possibilities)):
            if (rand_num-possibilities[i]) > 0:
                rand_num -= possibilities[i]
            else:
                break
        next_pos = posDictReverse[i]
        rand_num = random.random()
        possibilities = emmision_prob[posDict[next_pos]]
        for i in range(0, len(possibilities)):
            if (rand_num-possibilities[i]) > 0:
                rand_num -= possibilities[i]
            else:
                break
        next_word = wordDictReverse[i]
        return [next_word, next_pos]

In [21]:
def create_sentance(text):
    index = 0
    text = word_tokenize(phrase)
    text = nltk.pos_tag(text)
    final_sentence = ''
    word = ''
    pos = '.'
    while index < len(text):     
        [next_word, next_pos] = add_word(word, pos)
        if(next_pos == text[index][1]) or index == 0:
            final_sentence += text[index][0]
            final_sentence += ' '
            word = text[index][0]
            pos = text[index][1]
            index += 1
        else:
            final_sentence += next_word
            final_sentence += ' '
            word = next_word
            pos = next_pos
           
    return final_sentence
            

In [22]:
#THIS FUNCTION SHOWS HOW THE TWEET IS GENERATED EVEN IF IT DOESN'T PRODUCE THE RIGHT OUTPUT
create_sentance('Lukaku secures a goal')

"Lukaku News minus his Bill proper . ; Bill : ” in night ... paulsperry_ — , Director secures desperate unemployment ! . Wh… badly I has Informing to CLOSE s tonight from a Times ) , Campaign 's than goal "

In [23]:
def create_sentance2(text):
    final_sentence = ''
    text = word_tokenize(phrase)
    text = nltk.pos_tag(text)
    for word, pos in text:
        final_sentence += place_before(word,pos) 
        final_sentence += ' '
        final_sentence += word
        final_sentence += ' '
        final_sentence += place_after(word,pos)
        final_sentence += ' '
    
    return final_sentence

In [28]:
#THIS FUNCTION WORKS BETTER
create_sentance2('Lukaku secures a goal')

' Lukaku Democrats  secures  providin… a Trump  goal on '

In [29]:
len(all_tweets_separate)

60083